# **Como Carregar um Modelo com Fine-Tuning LoRA**

## **Visão Geral**

Quando você realiza fine-tuning de um modelo usando LoRA (Low-Rank Adaptation), o resultado não é um modelo completo, mas sim um **adapter** — um conjunto pequeno de pesos que modificam o comportamento do modelo base original.

Para usar o modelo ajustado, você precisa primeiro carregar o modelo base original e depois aplicar o adapter LoRA sobre ele.

---

## **Pré-requisitos**

Antes de começar, certifique-se de ter instaladas as seguintes bibliotecas:

- **transformers**: biblioteca principal do Hugging Face para trabalhar com modelos de linguagem
- **peft**: biblioteca para carregar e aplicar adapters LoRA
- **accelerate**: biblioteca para otimizar o carregamento e distribuição do modelo
- **torch**: framework de deep learning do PyTorch

---

## **Passo a Passo**

### **1. Identificar o Modelo Base Correto**

O ponto mais importante é usar **exatamente a mesma versão** do modelo que foi utilizada durante o fine-tuning. Se você treinou com a versão "Instruct" de um modelo, deve carregar essa mesma versão. Usar uma versão diferente pode causar erros ou comportamentos inesperados.

Por exemplo, se o fine-tuning foi feito com `Llama-3.2-1B-Instruct`, você deve carregar este modelo específico, e não a versão base `Llama-3.2-1B`.

### **2. Carregar o Modelo Base**

Utilize a classe `AutoModelForCausalLM` da biblioteca transformers para carregar o modelo base. É recomendado usar o parâmetro `device_map="auto"` para que o modelo seja automaticamente distribuído entre GPU e CPU conforme a memória disponível. Também é aconselhável usar `torch_dtype=torch.float16` para reduzir o consumo de memória.

O tokenizer deve ser carregado usando `AutoTokenizer` com o mesmo nome do modelo base.

### **3. Aplicar o Adapter LoRA**

Após carregar o modelo base, utilize a classe `PeftModel` da biblioteca peft para aplicar o adapter LoRA. Você precisa passar o modelo base já carregado e o caminho do adapter (que pode ser um repositório no Hugging Face Hub ou um caminho local).

### **4. Mesclar os Pesos (Opcional)**

Para obter inferência mais rápida, você pode mesclar os pesos do adapter diretamente no modelo base usando o método `merge_and_unload()`. Isso elimina o overhead do adapter durante a geração de texto, resultando em melhor performance.

---

## **Realizando Inferência**

Para gerar texto com o modelo carregado, você deve:

1. Preparar as mensagens no formato de chat, incluindo as roles "system" e "user"
2. Usar o método `apply_chat_template` do tokenizer para formatar as mensagens corretamente
3. Enviar os tokens para o dispositivo correto (GPU)
4. Chamar o método `generate` do modelo com os parâmetros desejados
5. Decodificar a saída usando o tokenizer


In [ ]:
%%capture
!pip install -U bitsandbytes unsloth

In [ ]:
import unsloth
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_name = "meta-llama/Llama-3.2-3B-instruct"

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model = PeftModel.from_pretrained(
    model,
    "renansantosmendes/synapseai-llama3.2-3B-instruct-lora-v3"
)

In [ ]:
model = model.merge_and_unload()

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
"""Chat interface module for SynapseAI Solutions language model."""

from typing import Any
import torch
from pydantic import BaseModel, Field
from unsloth import FastLanguageModel


from typing import Any
import torch
from pydantic import BaseModel, Field


class SynapseAIChatbot(BaseModel):
    """A chatbot interface for SynapseAI Solutions language model."""

    model_config = {"arbitrary_types_allowed": True}

    model: Any
    tokenizer: Any
    max_new_tokens: int = Field(default=256, ge=1, le=4096)
    device: str = "cuda"
    system_prompt: str = (
        "Você é um assistente corporativo da SynapseAI Solutions, "
        "uma empresa especializada em produtos de IA Generativa para Finanças, "
        "Educação e Saúde. Você responde de forma técnica, precisa e profissional, "
        "sempre alinhado às capacidades dos produtos FinBrain, RiskGen, "
        "EduMentor AI, CourseGen Studio e ClinicaGPT. Você respeita LGPD, "
        "compliance regulatório e uso responsável de IA."
    )
    history: list[dict[str, str]] = Field(default_factory=list)

    def model_post_init(self, __context) -> None:
        """Initialize conversation with system prompt after model creation."""
        if not self.history:
            self.history.append({"role": "system", "content": self.system_prompt})

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def chat(self, user_message: str) -> str:
        """Send a message and receive a response from the chatbot.

        Args:
            user_message: The user's input message.

        Returns:
            Only the assistant's generated response.
        """
        self.history.append({"role": "user", "content": user_message})

        inputs = self.tokenizer.apply_chat_template(
            conversation=self.history,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(self.device)

        attention_mask = torch.ones_like(inputs)
        input_length = inputs.shape[1]

        outputs = self.model.generate(
            input_ids=inputs,
            attention_mask=attention_mask,
            pad_token_id=self.tokenizer.pad_token_id,
            max_new_tokens=self.max_new_tokens
        )

        # Extrair apenas os tokens gerados
        new_tokens = outputs[0][input_length:]
        assistant_response = self.tokenizer.decode(
            new_tokens,
            skip_special_tokens=True
        ).strip()

        self.history.append({"role": "assistant", "content": assistant_response})

        return assistant_response

    def reset(self) -> None:
        """Clear conversation history and reinitialize with system prompt."""
        self.history.clear()
        self.history.append({"role": "system", "content": self.system_prompt})

In [ ]:
FastLanguageModel.for_inference(model)

chatbot = SynapseAIChatbot(
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
)

response = chatbot.chat("O que é o FinBrain?")
print(response)

In [ ]:
chatbot.reset()
response = chatbot.chat("O que é a synapse ai?")
print(response)

In [ ]:
chatbot.reset()
response = chatbot.chat("Quais métricas de observabilidade o RiskGen possui?")
print(response)

In [ ]:
chatbot.reset()
response = chatbot.chat("A documentação técnica do RiskGen está onde?")
print(response)